In [5]:
%env SM_FRAMEWORK=tf.keras

#import segmentation_models as sm
import cv2
#from skimage.io import imread, imsave
import numpy as np
import glob
import os

#from sklearn.metrics import jaccard_score


env: SM_FRAMEWORK=tf.keras


In [6]:
def dice(y_pred, y_true, k = 1):
    intersection = np.sum(y_pred[y_true==k]) * 2.0
    dice = intersection / (np.sum(y_pred) + np.sum(y_true))
    return dice


In [7]:
#Syntax: cv2.imread(path, flag)
#
#Parameters:
#path: A string representing the path of the image to be read.
#flag: It specifies the way in which image should be read. It’s default value is cv2.IMREAD_COLOR
#
#Return Value: This method returns an image that is loaded from the specified file.

#cv2.IMREAD_COLOR: It specifies to load a color image.
#    Any transparency of image will be neglected. It is the default flag.
#    Alternatively, we can pass integer value 1 for this flag.
#cv2.IMREAD_GRAYSCALE: It specifies to load an image in grayscale mode.
#    Alternatively, we can pass integer value 0 for this flag.
#cv2.IMREAD_UNCHANGED: It specifies to load an image as such including alpha channel.
#    Alternatively, we can pass integer value -1 for this flag.

def img_pr_mask_adjust(img):
    img = img / 255.0
    img = np.array(img)

    #print(img.shape)
    #img = img[:,:, 0] # imread IMREAD_GRAYSCALE
    #print(img.shape)

    img[img >= 0.5] = 1
    img[img < 1] = 0
    img = img.astype('uint8')

    return img


In [8]:
#pathStart="/run/media/j/tmp_ntfs/0_t_mest_bkp/tests/Segmentacao Linha Plantio CNN/"

pathStart="E:/Joao/testDice Segmentacao Linha Plantio CNN Backes/"

#---------------------------------------------------------------------------------------------
# Result to dataset A, B, C e D
#test_result= 01 to 12, A to D

#test_result="01A"
test_result="02C"

diretorio_pr = pathStart + "Resultados/" + "Teste_" + test_result + "/result-images/"

#---------------------------------------------------------------------------------------------
# Result to dataset E
#test_result= 01 e 09, E

#test_result="01E"
#test_result="09E"

#diretorio_pr = pathStart + "Teste_" + test_result + "/result-images/"
#---------------------------------------------------------------------------------------------

print(diretorio_pr)

for i in range(4): #
    if i == 0:
        dataset = 'A'
    elif i == 1:
        dataset = 'B'
    elif i == 2:
        dataset = 'C'
    elif i == 3:
        dataset = 'D'

    print("\ndataset:", dataset)

    diretorio_mask= pathStart + "Recortes/Base_" + dataset + "_mask/"

    dice_score_all = 0
    count = 0

    for name_pr in sorted(glob.glob(diretorio_pr + "/" + dataset + "*.png")):
        #print(name_pr)

        img_pr = cv2.imread(name_pr, cv2.IMREAD_GRAYSCALE)

        #name_mask = name_pr.split('/')[-1]
        name_mask = name_pr.split('\\')[-1] # Windows strange path
        #print(name_mask)
        #print(diretorio_mask + name_mask)

        img_mask = cv2.imread(diretorio_mask + name_mask, cv2.IMREAD_GRAYSCALE)

        #print("Image predict path:", name_pr)
        #print("Image mask path:", diretorio_mask + name_mask)

        #print(img_pr)
        #print(img_mask)

        test = 3
#=== Test 1 - dice_score avg: 0.9553537285052939 - round 2 - 0.96
        if test == 1:
            img_pr = img_pr / 255

            img_mask = img_mask / 255

#=== Test 2 - dice_score avg: 0.8923372642470235 - round 2 - 0.89
        elif test == 2:
            img_pr = img_pr / 255
            img_pr = img_pr.astype('uint8')

            img_mask = img_mask / 255
            img_mask = img_mask.astype('uint8')

#=== Test 3 - dice_score avg: 0.9644982330072099 - round 2 - 0.96
        elif test == 3: # best result
            #img_pr = cv2.imread(name_pr)
            #img_mask = cv2.imread(diretorio_mask + name_mask)

            img_pr = img_pr_mask_adjust(img_pr)
            img_mask = img_pr_mask_adjust(img_mask)
#=== Test 4 - dice_score avg: 0.9091327108048295 - round 2 - 0.91
        elif test == 4: # Used in "paper segmentation" - of 3 nets
            # get a the img "as" RGB from model predit
            img_pr = cv2.imread(name_pr)

            img_pr = np.array(img_pr)
            img_pr = img_pr[:,:, 0]
            img_pr[img_pr >= 0.5] = 1
            img_pr[img_pr < 1] = 0
            img_pr = img_pr.astype('uint8')

            img_mask[img_mask > 0] = 1
#=========

        #img_pr = img_pr / 255
        #img_pr = img_pr.astype('uint8')
    
        #img_mask = img_mask / 255
        #img_mask = img_mask.astype('uint8')

        #img_pr = img_pr_mask_adjust(img_pr)
        #img_mask = img_pr_mask_adjust(img_mask)
        
        dice_score = dice(img_pr, img_mask, k = 1) # k can be 255 or 1
        #print ("Dice Similarity {} : {}".format(name_mask, dice_score))
        #input()
    
        dice_score_all += dice_score
        count += 1

    print("count:", count)
    print("dice_score_all:", dice_score_all)
    print("dice_score avg:", dice_score_all/count)
    print("dice_score avg round:", round(dice_score_all/count, 2))

print("Done all!")

print("\ntest value:", test)

E:/Joao/testDice Segmentacao Linha Plantio CNN Backes/Resultados/Teste_02C/result-images/

dataset: A
count: 678
dice_score_all: 593.1028545330705
dice_score avg: 0.8747829712877147
dice_score avg round: 0.87

dataset: B
count: 3291
dice_score_all: 2989.1532120268284
dice_score avg: 0.9082811340099752
dice_score avg round: 0.91

dataset: C
count: 1552
dice_score_all: 1452.9419541350517
dice_score avg: 0.9361739395200075
dice_score avg round: 0.94

dataset: D
count: 2162
dice_score_all: 1702.301530288852
dice_score avg: 0.787373510771902
dice_score avg round: 0.79
Done all!

test value: 3
